In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Load CoverType from OpenML

In [3]:
X, y_mc = datasets.fetch_openml(data_id=180, return_X_y=True, as_frame=False)
X = X.astype(float)

In [4]:
# make it a binary classification problem for the sake of simplicity
y = y_mc == "Lodgepole_Pine"
xtrain, xtest, ytrain, ytest, ytrain_mc, ytest_mc = train_test_split(X, y, y_mc, test_size=0.2, shuffle=True)

# Train an XGBoost model

In [5]:
import veritas
import xgboost as xgb
import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score

In [6]:
params = {
    "n_estimators": 100,
    "eval_metric": "auc",
    "seed": 197,
    "max_depth": 10,
    "learning_rate": 0.2
}
model = xgb.XGBClassifier(**params)

t = time.time()
model.fit(X, y)
print(f"XGB trained in {time.time()-t} seconds")

XGB trained in 0.8577525615692139 seconds


In [7]:
ytrain_pred = model.predict(xtrain)
ytest_pred = model.predict(xtest)
acc_train = accuracy_score(ytrain, ytrain_pred)
acc_test = accuracy_score(ytest, ytest_pred)
auc_train = roc_auc_score(ytrain, ytrain_pred)
auc_test = roc_auc_score(ytest, ytest_pred)

print(f"Train acc: {acc_train:.3f}, test acc: {acc_test:.3f} wrt true labels")
print(f"Train auc: {auc_train:.3f}, test auc: {auc_test:.3f} wrt true labels")

Train acc: 0.945, test acc: 0.945 wrt true labels
Train auc: 0.946, test auc: 0.947 wrt true labels


# Generate adversarial examples

We only allow changes to the first 10 numerical attribute values. The remaining 44 attribute values are binary.

In [8]:
# FROM CLASS 0 -> CLASS 1
number_of_adv_examples = 10

rng = np.random.default_rng(seed=19656)
xtest0 = xtest[ytest==0, :]
subset = xtest0[rng.choice(range(xtest0.shape[0]), number_of_adv_examples), :]

eps = 0.2

In [9]:
at = veritas.get_addtree(model)
Xstd = xtrain.std(axis=0)
Xstd[10:] = 0.0 # allow no change in binary attributes


| XGBOOST's base_score
|   base_score diff std      5.40878462362659e-07 OK
|   base_score reported      0.46820667
|   versus manually detected -0.1273450059940551
|   abs err                  0.5955516759940551
|   rel err                  1.2719846045637393
|   (!) base_score NOT THE SAME with relative tolerance 0.001



In [10]:
adv_examples = []
for i in tqdm.tqdm(range(subset.shape[0])):
    base_example = subset[i, :]
    
    # allow each attribute to vary by eps*stddev of attribute to either side
    prune_box = [veritas.Interval(x-eps*s, x+eps*s) if s > 0.0
                 else veritas.Interval.constant(x)
                 for x, s in zip(base_example, Xstd)]
    
    config = veritas.Config(veritas.HeuristicType.MAX_OUTPUT)
    search = config.get_search(at, prune_box)

    # continue the search for at most 1 second
    # stop early when the optimal solution is found
    stop_reason = search.step_for(1.0, 1000)
    
    if search.num_solutions() > 0:
        sol = search.get_solution(0)
        adv_example = veritas.get_closest_example(sol, base_example, eps=1e-4)

        res = {"i": i, "adv_example": adv_example, "base_example": base_example}

        res["base_ypred_at"] = at.predict(np.atleast_2d(base_example))[0]
        res["base_ypred"] = model.predict_proba(np.atleast_2d(base_example))[0,1]
        res["adv_ypred"] = model.predict_proba(np.atleast_2d(adv_example))[0,1]
        res["adv_ypred_at"] = at.predict(np.atleast_2d(adv_example))[0]
        res["optimal"] = search.is_optimal()

        adv_examples.append(res)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.45it/s]


In [11]:
print("i  {:15}{:15}{:>8}{:>10}".format("p(y=1|normal)", "p(y=1|adv)", "success?", "optimal?"))
for res in adv_examples:
    i = res["i"]
    success = "y" if res["adv_ypred"]>0.5 else "-"
    optimal = "y" if res["optimal"] else "-"
    base = res["base_ypred"]
    adv = res["adv_ypred"]
    print("{:<3}{:>13}  {:>10}     {:>8}{:>10}".format(i, f'{base*100:4.1f}%', f'{adv*100:4.1f}%', success, optimal))

i  p(y=1|normal)  p(y=1|adv)     success?  optimal?
0          14.2%       43.6%            -         y
1          14.7%       55.3%            y         y
2           6.2%       86.4%            y         y
3           1.6%       74.7%            y         y
4           1.1%       44.2%            -         y
5          11.0%       79.1%            y         y
6           5.3%       67.6%            y         y
7           2.7%       36.8%            -         y
8          14.7%       93.9%            y         y
9          74.5%       99.1%            y         y


**Which attributes differ?**

> Name / Data Type / Measurement / Description
> --------------------------------------------
> Elevation / quantitative /meters / Elevation in meters  
> Aspect / quantitative / azimuth / Aspect in degrees azimuth  
> Slope / quantitative / degrees / Slope in degrees  
> Horizontal_Distance_To_Hydrology / quantitative / meters / Horz Dist to nearest surface water features  
> Vertical_Distance_To_Hydrology / quantitative / meters / Vert Dist to nearest surface water features  
> Horizontal_Distance_To_Roadways / quantitative / meters / Horz Dist to nearest roadway   
> Hillshade_9am / quantitative / 0 to 255 index / Hillshade index at 9am, summer solstice   
> Hillshade_Noon / quantitative / 0 to 255 index / Hillshade index at noon, summer solstice   
> Hillshade_3pm / quantitative / 0 to 255 index / Hillshade index at 3pm, summer solstice   
> Horizontal_Distance_To_Fire_Points / quantitative / meters / Horz Dist to nearest wildfire ignition points   
> Wilderness_Area (4 binary columns) / qualitative / 0 (absence) or 1 (presence) / Wilderness area designation   
> Soil_Type (40 binary columns) / qualitative / 0 (absence) or 1 (presence) / Soil Type designation   
> Cover_Type (7 types) / integer / 1 to 7 / Forest Cover Type designation 

In [12]:
import pprint
attribute_names = ["elevation", "aspect", "slope", "hoz_dist_hydro", "ver_dist_hydro", "hoz_dist_road", "shade_9am", "shade_noon", "shade_3am", "hoz_dist_fire"]# + [f"wilderness{k}" for k in range(4)] + [f"soil{k}" for k in range(40)]

for adv in adv_examples:
    i = adv["i"]
    base_ex, adv_ex = adv["base_example"], adv_examples[i]["adv_example"]
    print(f'i={i} p(y=1) = {adv["base_ypred"]*100:.1f}% -> {adv["adv_ypred"]*100:.1f}%')
    print("    {:15}{:>10}{:>10}{:>10}".format("attribute", "normal", "adv.", "diff."))
    for j, attrname in enumerate(attribute_names):
        b, a = base_ex[j], adv_ex[j]
        print("    {:15}{:>10}{:>10}{:>10}".format(attrname, f'{b:.1f}', f'{a:.1f}', f'{a-b:.1f}'))

i=0 p(y=1) = 14.2% -> 43.6%
    attribute          normal      adv.     diff.
    elevation          2806.0    2840.0      34.0
    aspect              185.0     163.0     -22.0
    slope                 6.0       6.0      -0.0
    hoz_dist_hydro      120.0      85.0     -35.0
    ver_dist_hydro       18.0      29.0      11.0
    hoz_dist_road      1127.0     967.0    -160.0
    shade_9am           221.0     221.0       0.0
    shade_noon          244.0     245.0       1.0
    shade_3am           158.0     158.0       0.0
    hoz_dist_fire      1124.0    1383.0     259.0
i=1 p(y=1) = 14.7% -> 55.3%
    attribute          normal      adv.     diff.
    elevation          2952.0    2915.0     -37.0
    aspect              270.0     263.0      -7.0
    slope                 9.0       9.0      -0.0
    hoz_dist_hydro       30.0      67.0      37.0
    ver_dist_hydro       -1.0      10.0      11.0
    hoz_dist_road       330.0     433.0     103.0
    shade_9am           197.0     197.0     